tobac example: Compute bulk statistics as a postprocessing step
=== 

Instead of during the feature detection or segmentation process, you can also calculate bulk statistics of your detected/tracked objects as a postprocessing step, i.e. based on a segmentation mask. This makes it possible to combine different datasets and derive statistics for your detected features based on other input fields (e.g., get precipitation statistics under cloud features that were segmented based on brightness temperatures or outgoing longwave radiation). 

This notebook shows an example for how to compute bulk statistics for detected features as a postprocessing step, that is based on the segmentation mask that we have already created. We perform the feature detection and segmentation with data from [our example for precipitation tracking](https://github.com/tobac-project/tobac/blob/main/examples/Example_Precip_Tracking/Example_Precip_Tracking.ipynb). 

In [1]:
# Import libraries
import iris
import numpy as np
import pandas as pd
import xarray as xr 
import matplotlib.pyplot as plt
import datetime
import shutil
from six.moves import urllib
from pathlib import Path
%matplotlib inline

In [3]:
# Import tobac itself
import tobac
print('using tobac version', str(tobac.__version__))

using tobac version 1.5.3


In [4]:
# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

**Feature detection** 

In [5]:
#Set up directory to save output:
savedir=Path("Save")
if not savedir.is_dir():
    savedir.mkdir()
plot_dir=Path("Plot")
if not plot_dir.is_dir():
    plot_dir.mkdir()

In [6]:
data_out=Path('../')
# Download the data: This only has to be done once for all tobac examples and can take a while
data_file = list(data_out.rglob('data/Example_input_Precip.nc'))
if len(data_file) == 0:
    file_path='https://zenodo.org/records/3195910/files/climate-processes/tobac_example_data-v1.0.1.zip'
    #file_path='http://zenodo..'
    tempfile=Path('temp.zip')
    print('start downloading data')
    request=urllib.request.urlretrieve(file_path, tempfile)
    print('start extracting data')
    shutil.unpack_archive(tempfile, data_out)
    tempfile.unlink()
    print('data extracted')
    data_file = list(data_out.rglob('data/Example_input_Precip.nc'))

start downloading data
start extracting data
data extracted


In [7]:
Precip=iris.load_cube(str(data_file[0]),'surface_precipitation_average')

In [8]:
parameters_features={}
parameters_features['position_threshold']='weighted_diff'
parameters_features['sigma_threshold']=0.5
parameters_features['min_distance']=0
parameters_features['sigma_threshold']=1
parameters_features['threshold']=[1,2,3,4,5,10,15] #mm/h
parameters_features['n_erosion_threshold']=0
parameters_features['n_min_threshold']=3

# get temporal and spation resolution of the data
dxy,dt=tobac.get_spacings(Precip)

In [9]:
# Feature detection based on based on surface precipitation field and a range of thresholds
print('starting feature detection based on multiple thresholds')
Features= tobac.feature_detection_multithreshold(Precip,dxy,**parameters_features) 
print('feature detection done')
Features.to_hdf(savedir / 'Features.h5','table')
print('features saved')

starting feature detection based on multiple thresholds
feature detection done
features saved


features saved


**Segmentation** 

In [10]:
# Dictionary containing keyword arguments for segmentation step:
parameters_segmentation={}
parameters_segmentation['method']='watershed'
parameters_segmentation['threshold']=1  # mm/h mixing ratio

# get temporal and spation resolution of the data
dxy,dt=tobac.get_spacings(Precip)

In [11]:
# Perform Segmentation and save resulting mask to NetCDF file:
print('Starting segmentation based on surface precipitation')
Mask_Precip,Features_Precip=tobac.segmentation_2D(Features,Precip,dxy,**parameters_segmentation)
print('segmentation based on surface precipitation performed, start saving results to files')
iris.save([Mask_Precip], savedir / 'Mask_segmentation_precip.nc', zlib=True, complevel=4)                
Features_Precip.to_hdf(savedir / 'Features_Precip.h5', 'table')
print('segmentation surface precipitation performed and saved')

Starting segmentation based on surface precipitation
segmentation based on surface precipitation performed, start saving results to files
segmentation surface precipitation performed and saved


segmentation surface precipitation performed and saved


**Get bulk statistics from segmentation mask file**

You can decide which statistics to calculate by providing a dictionary with the name of the metric as keys (this will be the name of the column added to the dataframe) and functions as values. Note that it is also possible to provide input parameter to these functions. 

In [12]:
from tobac.utils import get_statistics_from_mask

#### Defining the dictionary for the statistics to be calculated 

In [13]:
statistics = {}
statistics['mean_precip'] = np.mean
statistics['total_precip'] = np.sum
statistics['max_precip'] = np.max

For some functions, we need to provide additional input parameters, e.g. [np.percentile()](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html). These can be provided as key word arguments in form of a dictionary. So instead of the function, you can provide a tuple with both the function and its respective input parameters: 


In [14]:
statistics['percentiles'] = (np.percentile, {'q': [95,99]})

In [22]:
Precip

Surface Precipitation Average (mm h-1),time,south_north,west_east
Shape,47,198,198
Dimension coordinates,,,
time,x,-,-
south_north,-,x,-
west_east,-,-,x
Auxiliary coordinates,,,
projection_y_coordinate,-,x,-
y,-,x,-
latitude,-,x,x
longitude,-,x,x


In [27]:
features = Features_Precip.drop('projection_x_coordinate', axis=1)
features = features.drop('projection_y_coordinate', axis=1)
features = features.drop('south_north', axis=1)
features = features.drop('west_east', axis=1)
features = features.drop('y_0', axis=1)
features = features.drop('x_0', axis=1)
features

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,ncells
0,0,1,50.065727,139.857477,9,1,1,2013-06-19 20:05:00,2013-06-19 20:05:00,29.846362,-94.172015,10
1,0,15,120.527119,172.500325,4,1,2,2013-06-19 20:05:00,2013-06-19 20:05:00,30.166929,-93.996892,10
2,0,18,126.779273,145.368401,15,1,3,2013-06-19 20:05:00,2013-06-19 20:05:00,30.196499,-94.139960,11
3,0,34,111.611369,155.452030,4,2,4,2013-06-19 20:05:00,2013-06-19 20:05:00,30.126871,-94.087317,19
4,0,35,111.765231,164.938866,8,2,5,2013-06-19 20:05:00,2013-06-19 20:05:00,30.127221,-94.037226,20
...,...,...,...,...,...,...,...,...,...,...,...,...
1124,46,90,163.334480,35.049366,47,15,1125,2013-06-19 23:55:00,2013-06-19 23:55:00,30.365944,-94.722384,185
1125,46,91,177.119093,1.938903,50,15,1126,2013-06-19 23:55:00,2013-06-19 23:55:00,30.429151,-94.897570,162
1126,46,92,183.889436,100.054108,36,15,1127,2013-06-19 23:55:00,2013-06-19 23:55:00,30.458648,-94.377737,76
1127,46,93,196.870749,177.405664,10,15,1128,2013-06-19 23:55:00,2013-06-19 23:55:00,30.515366,-93.967334,45


In [32]:
features

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,ncells
0,0,1,50.065727,139.857477,9,1,1,2013-06-19 20:05:00,2013-06-19 20:05:00,29.846362,-94.172015,10
1,0,15,120.527119,172.500325,4,1,2,2013-06-19 20:05:00,2013-06-19 20:05:00,30.166929,-93.996892,10
2,0,18,126.779273,145.368401,15,1,3,2013-06-19 20:05:00,2013-06-19 20:05:00,30.196499,-94.139960,11
3,0,34,111.611369,155.452030,4,2,4,2013-06-19 20:05:00,2013-06-19 20:05:00,30.126871,-94.087317,19
4,0,35,111.765231,164.938866,8,2,5,2013-06-19 20:05:00,2013-06-19 20:05:00,30.127221,-94.037226,20
...,...,...,...,...,...,...,...,...,...,...,...,...
1124,46,90,163.334480,35.049366,47,15,1125,2013-06-19 23:55:00,2013-06-19 23:55:00,30.365944,-94.722384,185
1125,46,91,177.119093,1.938903,50,15,1126,2013-06-19 23:55:00,2013-06-19 23:55:00,30.429151,-94.897570,162
1126,46,92,183.889436,100.054108,36,15,1127,2013-06-19 23:55:00,2013-06-19 23:55:00,30.458648,-94.377737,76
1127,46,93,196.870749,177.405664,10,15,1128,2013-06-19 23:55:00,2013-06-19 23:55:00,30.515366,-93.967334,45


In [41]:
precip.shape

(47, 198, 198)

In [53]:
precip = xr.DataArray.from_iris(Precip)
precip.latitude[10::10] = np.nan 
precip.longitude[10::10] = np.nan 

In [47]:
features_with_stats = get_statistics_from_mask(features, Mask_Precip, precip, statistic=statistics)

### Look at the output: 

In [48]:
features_with_stats.mean_precip.head()

0    1.629695
1    1.409547
2    2.441526
3    1.938501
4    2.486886
Name: mean_precip, dtype: object

In [49]:
features_with_stats.total_precip.head()

0    16.296951
1    14.095468
2    26.856783
3    36.831512
4    49.737709
Name: total_precip, dtype: object

In [50]:
features_with_stats.percentiles.head()

0      ([2.221776068210602, 2.276183712482452],)
1    ([1.8030404090881347, 1.8164567756652832],)
2      ([3.710712432861328, 3.759503173828125],)
3      ([3.940941762924194, 4.042321195602417],)
4     ([4.087516045570374, 4.3222578477859495],)
Name: percentiles, dtype: object